In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
# uncomment this to read and process all excel files again

tennis_df = pd.DataFrame()
for year in range(2001, 2022):
    
    file = ""
    if year < 2013:
        file = f"{year}.xls"
        print(file)
    else:
        assert(year >= 2013)
        file = f"{year}.xlsx"
        print(file)
    
    year_df = pd.read_excel(file)
    year_df['Year'] = year
    tennis_df = tennis_df.append(year_df)


2001.xls
2002.xls
2003.xls
2004.xls
2005.xls
2006.xls
2007.xls
2008.xls
2009.xls
2010.xls
2011.xls
2012.xls
2013.xlsx


C:\Users\briel\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


2014.xlsx
2015.xlsx
2016.xlsx
2017.xlsx
2018.xlsx
2019.xlsx
2020.xlsx
2021.xlsx


In [3]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,CBW,CBL,GBW,GBL,IWW,IWL,SBW,SBL,Year,B365W,B365L,B&WW,B&WL,EXW,EXL,PSW,PSL,WPts,LPts,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Clement A.,Gaudenzi A.,18.0,101,6.0,7.0,6.0,0.0,6.0,3.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Goldstein P.,Jones A.,81.0,442,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Haas T.,Smith L.,23.0,485,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,1.12,4.0,1.1,3.5,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Henman T.,Rusedski G.,10.0,69,6.0,2.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Hewitt L.,Arthurs W.,7.0,83,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.28,3.3,1.12,3.8,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# file_name = "tennis_data_2001_2021_raw.xlsx"
# tennis_df.to_excel(file_name) 


In [5]:
# tennis_df = pd.read_excel(file_name)

In [6]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,CBW,CBL,GBW,GBL,IWW,IWL,SBW,SBL,Year,B365W,B365L,B&WW,B&WL,EXW,EXL,PSW,PSL,WPts,LPts,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Clement A.,Gaudenzi A.,18.0,101,6.0,7.0,6.0,0.0,6.0,3.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Goldstein P.,Jones A.,81.0,442,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Haas T.,Smith L.,23.0,485,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,1.12,4.0,1.1,3.5,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Henman T.,Rusedski G.,10.0,69,6.0,2.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,3,Hewitt L.,Arthurs W.,7.0,83,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.28,3.3,1.12,3.8,NaN,NaN,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# only want to look at completed matches
print(len(tennis_df))
tennis_df = tennis_df[tennis_df['Comment']=='Completed']
print(len(tennis_df))

55604
53567


In [8]:
# as of right now, don't see these columns as adding much to the model/data
# once we can get this to work with BET365 then we can look at other books
cols_list_drop = ['Best of',
       'Comment', 'CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL',
        'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL',
       'WPts', 'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL', 'MaxW',
       'MaxL', 'AvgW', 'AvgL']

In [9]:
tennis_df = tennis_df.drop(cols_list_drop, axis=1)

In [10]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Year,B365W,B365L
0,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Clement A.,Gaudenzi A.,18.0,101,6.0,7.0,6.0,0.0,6.0,3.0,NaN,NaN,NaN,NaN,2.0,1.0,2001,NaN,NaN
1,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Goldstein P.,Jones A.,81.0,442,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2001,NaN,NaN
2,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Haas T.,Smith L.,23.0,485,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2001,NaN,NaN
3,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Henman T.,Rusedski G.,10.0,69,6.0,2.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2001,NaN,NaN
4,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Hewitt L.,Arthurs W.,7.0,83,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2001,NaN,NaN


In [11]:
tennis_df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1', 'W2', 'L2',
       'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets', 'Year', 'B365W',
       'B365L'],
      dtype='object')

In [12]:
len(tennis_df)

53567

In [13]:
# for year, frame in tennis_df.groupby('Year'):
#     percent_missing = frame.isnull().sum() * 100 / len(frame)
#     missing_value_df = pd.DataFrame({'column_name': frame.columns,
#                                  'percent_missing': percent_missing})
#     print(year)
#     display(missing_value_df)

In [14]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Year,B365W,B365L
0,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Clement A.,Gaudenzi A.,18.0,101,6.0,7.0,6.0,0.0,6.0,3.0,NaN,NaN,NaN,NaN,2.0,1.0,2001,NaN,NaN
1,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Goldstein P.,Jones A.,81.0,442,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2001,NaN,NaN
2,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Haas T.,Smith L.,23.0,485,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2001,NaN,NaN
3,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Henman T.,Rusedski G.,10.0,69,6.0,2.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2001,NaN,NaN
4,1,Adelaide,AAPT Championships,2001-01-01,International,Outdoor,Hard,1st Round,Hewitt L.,Arthurs W.,7.0,83,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2001,NaN,NaN


In [15]:
tennis_df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1', 'W2', 'L2',
       'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets', 'Year', 'B365W',
       'B365L'],
      dtype='object')

In [16]:
# dropping all na's because the only NA's were in B365 and in rank - can't run model without either of these
tennis_df = tennis_df.dropna(
    subset=['ATP', 'Location', 'Tournament', 'Date', 'Series', 
    'Court', 'Surface', 'Round', 'Winner', 'Loser', 'WRank', 
    'LRank', 'Wsets', 'Lsets', 'Year', 'B365W', 'B365L']
    )

In [17]:
# make sure that the drop na worked
percent_missing = tennis_df.isnull().sum() * 100 / len(tennis_df)
missing_value_df = pd.DataFrame({'column_name': tennis_df.columns,
                             'percent_missing': percent_missing})
display(missing_value_df)

,column_name,percent_missing
ATP,ATP,0.000000
Location,Location,0.000000
Tournament,Tournament,0.000000
Date,Date,0.000000
Series,Series,0.000000
Court,Court,0.000000
Surface,Surface,0.000000
Round,Round,0.000000
Winner,Winner,0.000000
Loser,Loser,0.000000


In [18]:
len(tennis_df)

47925

In [19]:
tennis_df.reset_index(inplace=True, drop=True)

In [20]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Year,B365W,B365L
0,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Johansson T.,Pretzsch A.,7.0,133,7.0,6.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,1.100,6.500
1,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Kafelnikov Y.,Bjorkman J.,6.0,56,7.0,6.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,1.444,2.625
2,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Koubek S.,Moya C.,41.0,20,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,2.625,1.444
3,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Pavel A.,Stepanek R.,21.0,150,7.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,1.533,2.375
4,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Sampras P.,Stoliarov A.,13.0,107,4.0,6.0,6.0,3.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,2002,1.143,5.000


In [21]:
# convert ranks to ints
tennis_df['WRank'] = tennis_df['WRank'].astype(int)
tennis_df['LRank'] = tennis_df['LRank'].astype(int)

In [22]:
tennis_df.dtypes

ATP                    int64
Location              object
Tournament            object
Date          datetime64[ns]
Series                object
Court                 object
Surface               object
Round                 object
Winner                object
Loser                 object
WRank                  int32
LRank                  int32
W1                   float64
L1                   float64
W2                    object
L2                    object
W3                    object
L3                    object
W4                   float64
L4                   float64
W5                   float64
L5                   float64
Wsets                float64
Lsets                float64
Year                   int64
B365W                float64
B365L                float64
dtype: object

In [23]:
# this code identifies which tournaments don't have semifinals or finals

unique_tournaments = tennis_df['Tournament'].unique()
unique_rounds = tennis_df['Round'].unique()
display(unique_rounds)

def find_no_rounds_available(tournaments_list, round_list):

    for tournament in tournaments_list:

        tournament_df = tennis_df[tennis_df['Tournament']==tournament]


        if tournament == "Queens Club":
            display(tournament_df)

        for round_num in round_list:

            round_df = tournament_df[tournament_df['Round']==round_num]

            if len(round_df)==0:
                print(tournament, f"NO {round_num}")


find_no_rounds_available(
    tournaments_list=unique_tournaments, 
    round_list=['Semifinals', 'The Final'])

array(['1st Round', '2nd Round', '3rd Round', '4th Round',
       'Quarterfinals', 'Semifinals', 'The Final', 'Round Robin'],
      dtype=object)

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Year,B365W,B365L
7,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Godwin N.,Gasquet R.,111,281,7.0,6.0,5.0,7.0,6.0,3.0,NaN,NaN,NaN,NaN,2.0,1.0,2002,1.533,2.375
8,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Lee H.T.,Larsson M.,117,142,6.0,3.0,6.0,7.0,7.0,6.0,NaN,NaN,NaN,NaN,2.0,1.0,2002,2.625,1.444
9,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Morrison J.,Ilie A.,98,176,6.0,3.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,1.571,2.250
10,33,Queens Club,Queens Club,2002-06-10,International,Outdoor,Grass,1st Round,Spadea V.,Llodra M.,73,103,7.0,6.0,7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,2.000,1.727


Queens Club NO Semifinals
Queens Club NO The Final
Zhuhai Open NO Semifinals


In [24]:
queens_club = tennis_df[tennis_df['Tournament']=='Queens Club']
# display('Queens Club', queens_club)
display("Queens club", queens_club['Round'].unique())
zhuhai = tennis_df[tennis_df['Tournament']=='Zhuhai Open']
# display('Zhuhai Open', zhuhai)
display("Zhuhai", zhuhai['Round'].unique())

'Queens club'

array(['1st Round'], dtype=object)

'Zhuhai'

array(['1st Round', '2nd Round', 'Quarterfinals', 'The Final'],
      dtype=object)

In [25]:
tennis_df[tennis_df['Tournament']=='Zhuhai Open'].head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Year,B365W,B365L
44062,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Norrie C.,Gojowczyk P.,68,92,6.0,1.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2019,1.53,2.37
44063,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Ramos-Vinolas A.,Nishioka Y.,46,60,6.0,2.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2019,2.75,1.40
44064,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Wu D.,Ito T.,339,141,3.0,6.0,6.0,3.0,7.0,5.0,NaN,NaN,NaN,NaN,2.0,1.0,2019,2.37,1.53
44065,56,Zhuhai,Zhuhai Open,2019-09-23,ATP250,Outdoor,Hard,1st Round,Mannarino A.,Zhang Ze.,61,333,7.0,6.0,7.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2019,1.28,3.50
44066,56,Zhuhai,Zhuhai Open,2019-09-24,ATP250,Outdoor,Hard,1st Round,Andujar P.,Johnson S.,53,94,2.0,6.0,6.0,2.0,6.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,2019,2.37,1.53


In [26]:
# dropping both of these tournaments from our set because they don't have both a semifinals and a finals
# we need both of these to measure how much pressure exists in a match

dropped_tournaments = ['Queens club', 'Zhuhai Open']

tennis_df = tennis_df[~tennis_df['Tournament'].isin(dropped_tournaments)]

assert(len(tennis_df[tennis_df['Tournament']=='Zhuhai Open'])==0)
assert(len(tennis_df[tennis_df['Tournament']=='Queens club'])==0)

In [27]:
# ranks final as highest pressure
# ranks semifinal as next highest
# categorizes everything else as not pressure

def tournament_rank_helper(round_tn):

    round_rank = None
    if round_tn == "The Final":
        round_rank = 1
    elif round_tn == "2nd Round":
        round_rank = 2
    else:
        round_rank = 3

    return round_rank


tennis_df['Tournament Rank'] = tennis_df['Round'].apply(tournament_rank_helper)


In [28]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Year,B365W,B365L,Tournament Rank
0,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Johansson T.,Pretzsch A.,7,133,7.0,6.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,1.100,6.500,3
1,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Kafelnikov Y.,Bjorkman J.,6,56,7.0,6.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,1.444,2.625,3
2,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Koubek S.,Moya C.,41,20,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,2.625,1.444,3
3,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Pavel A.,Stepanek R.,21,150,7.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,1.533,2.375,3
4,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Sampras P.,Stoliarov A.,13,107,4.0,6.0,6.0,3.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,2002,1.143,5.000,3


In [29]:
display(tennis_df['Round'].unique())

array(['1st Round', '2nd Round', '3rd Round', '4th Round',
       'Quarterfinals', 'Semifinals', 'The Final', 'Round Robin'],
      dtype=object)

In [30]:
# convert European odds into probabilities
# B365 factors in the VIM (how much the book makes)

tennis_df['B365W_prob'] = (1/tennis_df['B365W']).round(3)
tennis_df['B365L_prob'] = (1/tennis_df['B365L']).round(3)
tennis_df['B365_total_prob'] = tennis_df['B365W_prob'] + tennis_df['B365L_prob']
tennis_df['B365_diff_from_1'] = tennis_df['B365_total_prob'] - 1

# This takes VIM out to just get the real probabilities
tennis_df['real_prob_W'] = tennis_df['B365W_prob'] - (tennis_df['B365_diff_from_1']/2)
tennis_df['real_prob_L'] = tennis_df['B365L_prob'] - (tennis_df['B365_diff_from_1']/2)
tennis_df['real_total_prob'] = tennis_df['real_prob_W'] + tennis_df['real_prob_L']
tennis_df['real_diff_from_1'] = tennis_df['real_total_prob'] - 1

In [31]:
tennis_df.head(5)

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Year,B365W,B365L,Tournament Rank,B365W_prob,B365L_prob,B365_total_prob,B365_diff_from_1,real_prob_W,real_prob_L,real_total_prob,real_diff_from_1
0,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Johansson T.,Pretzsch A.,7,133,7.0,6.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,1.100,6.500,3,0.909,0.154,1.063,0.063,0.8775,0.1225,1.0,0.0
1,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Kafelnikov Y.,Bjorkman J.,6,56,7.0,6.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,1.444,2.625,3,0.693,0.381,1.074,0.074,0.6560,0.3440,1.0,0.0
2,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Koubek S.,Moya C.,41,20,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,2.625,1.444,3,0.381,0.693,1.074,0.074,0.3440,0.6560,1.0,0.0
3,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Pavel A.,Stepanek R.,21,150,7.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2002,1.533,2.375,3,0.652,0.421,1.073,0.073,0.6155,0.3845,1.0,0.0
4,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,1st Round,Sampras P.,Stoliarov A.,13,107,4.0,6.0,6.0,3.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,2002,1.143,5.000,3,0.875,0.200,1.075,0.075,0.8375,0.1625,1.0,0.0


In [32]:
# historical_reversed = tennis_df_historical.iloc[::-1]
# historical_reversed_copy = historical_reversed.copy()


In [33]:
# historical_reversed_copy.index.is_monotonic_decreasing
# tennis_df_historical.index.is_monotonic

In [34]:
drop_cols = [ 'Round', 'Year', 'B365W', 'B365L', 'B365W_prob', 'B365L_prob', 'B365_total_prob', 'B365_diff_from_1', 'real_total_prob', 'real_diff_from_1']

tennis_df = tennis_df.drop(drop_cols, axis=1)

In [35]:
import datetime

In [36]:
# creates player dictionary with basic info
def player_dict_generator(name, rank, prob):
    
    player_dict = {}
    player_dict['name'] = name
    player_dict['rank'] = rank
    player_dict['prob'] = prob
    
    return player_dict

In [37]:
season_dict = dict.fromkeys([12, 1, 2], 0)
season_dict.update(dict.fromkeys([3, 4, 5], 1))
season_dict.update(dict.fromkeys([6, 7, 8], 2))
season_dict.update(dict.fromkeys([9, 10, 11], 3))
season_dict

{12: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1, 6: 2, 7: 2, 8: 2, 9: 3, 10: 3, 11: 3}

In [38]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Tournament Rank,real_prob_W,real_prob_L
0,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Johansson T.,Pretzsch A.,7,133,7.0,6.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3,0.8775,0.1225
1,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Kafelnikov Y.,Bjorkman J.,6,56,7.0,6.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3,0.6560,0.3440
2,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Koubek S.,Moya C.,41,20,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3,0.3440,0.6560
3,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Pavel A.,Stepanek R.,21,150,7.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3,0.6155,0.3845
4,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Sampras P.,Stoliarov A.,13,107,4.0,6.0,6.0,3.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,3,0.8375,0.1625


In [39]:
for i in range(1,6):
    win_col = "W"+str(i)
    loss_col = "L"+str(i)


    # dataframe[column] = dataframe[column].replace(r'\s+', np.nan, regex=True)
    
    tennis_df[win_col] = tennis_df[win_col].replace(r'\s+', 0, regex=True)
    tennis_df[loss_col] = tennis_df[loss_col].replace(r'\s+', 0, regex=True)

    tennis_df[win_col] = tennis_df[win_col].astype(float)
    tennis_df[loss_col] = tennis_df[loss_col].astype(float)

In [40]:
def fill_nans_with_zero(df):

    # display missing vals cols
    print("Before Fill NA:")
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                'percent_missing': percent_missing})
    display(missing_value_df)

    values = {"W1": 0, 
    "L1": 0, 
    "W2": 0, 
    "L2": 0,
    "W3": 0,
    "L3": 0,
    "W4": 0,
    "L4": 0,
    "W5": 0,
    "L5": 0,
    }
    df = df.fillna(value=values)
    # df = df.fillna(0)

    print("After Fill NA:")
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                'percent_missing': percent_missing})
    display(missing_value_df)

    return df



In [41]:
tennis_df = fill_nans_with_zero(tennis_df)

Before Fill NA:


,column_name,percent_missing
ATP,ATP,0.000000
Location,Location,0.000000
Tournament,Tournament,0.000000
Date,Date,0.000000
Series,Series,0.000000
Court,Court,0.000000
Surface,Surface,0.000000
Winner,Winner,0.000000
Loser,Loser,0.000000
WRank,WRank,0.000000


After Fill NA:


,column_name,percent_missing
ATP,ATP,0.0
Location,Location,0.0
Tournament,Tournament,0.0
Date,Date,0.0
Series,Series,0.0
Court,Court,0.0
Surface,Surface,0.0
Winner,Winner,0.0
Loser,Loser,0.0
WRank,WRank,0.0


In [42]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Tournament Rank,real_prob_W,real_prob_L
0,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Johansson T.,Pretzsch A.,7,133,7.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,0.8775,0.1225
1,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Kafelnikov Y.,Bjorkman J.,6,56,7.0,6.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,0.6560,0.3440
2,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Koubek S.,Moya C.,41,20,6.0,3.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,0.3440,0.6560
3,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Pavel A.,Stepanek R.,21,150,7.0,6.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,0.6155,0.3845
4,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Sampras P.,Stoliarov A.,13,107,4.0,6.0,6.0,3.0,6.0,4.0,0.0,0.0,0.0,0.0,2.0,1.0,3,0.8375,0.1625


In [43]:
tennis_df.dtypes

ATP                         int64
Location                   object
Tournament                 object
Date               datetime64[ns]
Series                     object
Court                      object
Surface                    object
Winner                     object
Loser                      object
WRank                       int32
LRank                       int32
W1                        float64
L1                        float64
W2                        float64
L2                        float64
W3                        float64
L3                        float64
W4                        float64
L4                        float64
W5                        float64
L5                        float64
Wsets                     float64
Lsets                     float64
Tournament Rank             int64
real_prob_W               float64
real_prob_L               float64
dtype: object

In [44]:
def tennis_df_manipulation_helper(row):


    # set up information about the winner and loser
    winner = player_dict_generator(
        name=row['Winner'], 
        rank=row['WRank'], 
        prob=row['real_prob_W'])


    loser = player_dict_generator(
        name=row['Loser'], 
        rank=row['LRank'], 
        prob=row['real_prob_L'])


    # figure out the higher and lower rank
    higher_rank = {}
    lower_rank = {}
    if winner['rank'] < loser['rank']:
        higher_rank = winner
        lower_rank = loser
    else:
        higher_rank = loser
        lower_rank = winner

    row['HR_name'] = higher_rank['name']
    row['HR_rank'] = higher_rank['rank']
    row['HR_prob'] = higher_rank['prob']

    row['LR_name'] = lower_rank['name']
    row['LR_rank'] = lower_rank['rank']
    row['LR_prob'] = lower_rank['prob']

    hr_sets_won = None
    lr_sets_won = None
    hr_sets_lost = None
    lr_sets_lost = None

    hr_games_won = None
    lr_games_won = None
    hr_games_lost = None
    lr_games_lost = None

    winner_games_won = row['W1']+row['W2']+row['W3']+row['W4']+row['W5']
    loser_games_won = row['L1']+row['L2']+row['L3']+row['L4']+row['L5']
    # the winner lost the amount of games won by the winner, and viseversa
    winner_games_lost = loser_games_won
    loser_games_lost = winner_games_won

    if higher_rank['name']==winner['name']:
        assert(higher_rank['name']==winner['name'])
        assert(lower_rank['name']==loser['name'])
        # higher ranked won
        hr_sets_won = row['Wsets']
        hr_sets_lost = row['Lsets']
        lr_sets_won = row['Lsets']
        lr_sets_lost = row['Wsets']

        hr_games_won = winner_games_won
        hr_games_lost = winner_games_lost

        lr_games_won = loser_games_won
        lr_games_lost = loser_games_lost



    else:
        assert(higher_rank['name']==loser['name'])
        assert(lower_rank['name']==winner['name'])
        # lower ranked won, HR lost
        hr_sets_won = row['Lsets']
        hr_sets_lost = row['Wsets']
        lr_sets_won = row['Wsets']
        lr_sets_lost = row['Lsets']

        hr_games_won = loser_games_won
        hr_games_lost = loser_games_lost

        lr_games_won = winner_games_won
        lr_games_lost = winner_games_lost

    
    row['HR_Wsets'] = hr_sets_won
    row['LR_Wsets'] = lr_sets_won
    row['HR_Lsets'] = hr_sets_lost
    row['LR_Lsets'] = lr_sets_lost

    row['HR_Wgames'] = hr_games_won
    row['LR_Wgames'] = lr_games_won
    row['HR_Lgames'] = hr_games_lost
    row['LR_Lgames'] = lr_games_lost

    # time of year
    date_of_match = row['Date']
    match_month = date_of_match.month

    row['Match_szn'] = season_dict[match_month]

    return row

In [45]:
# tennis_df_copy = tennis_df.copy()
# test_tennis_df = tennis_df_copy[:20]
tennis_df = tennis_df.apply(tennis_df_manipulation_helper, axis=1)

In [46]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Tournament Rank,real_prob_W,real_prob_L,HR_name,HR_rank,HR_prob,LR_name,LR_rank,LR_prob,HR_Wsets,LR_Wsets,HR_Lsets,LR_Lsets,HR_Wgames,LR_Wgames,HR_Lgames,LR_Lgames,Match_szn
0,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Johansson T.,Pretzsch A.,7,133,7.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,0.8775,0.1225,Johansson T.,7,0.8775,Pretzsch A.,133,0.1225,2.0,0.0,0.0,2.0,13.0,6.0,6.0,13.0,2
1,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Kafelnikov Y.,Bjorkman J.,6,56,7.0,6.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,0.6560,0.3440,Kafelnikov Y.,6,0.6560,Bjorkman J.,56,0.3440,2.0,0.0,0.0,2.0,13.0,8.0,8.0,13.0,2
2,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Koubek S.,Moya C.,41,20,6.0,3.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,0.3440,0.6560,Moya C.,20,0.6560,Koubek S.,41,0.3440,0.0,2.0,2.0,0.0,6.0,12.0,12.0,6.0,2
3,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Pavel A.,Stepanek R.,21,150,7.0,6.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,0.6155,0.3845,Pavel A.,21,0.6155,Stepanek R.,150,0.3845,2.0,0.0,0.0,2.0,13.0,10.0,10.0,13.0,2
4,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Sampras P.,Stoliarov A.,13,107,4.0,6.0,6.0,3.0,6.0,4.0,0.0,0.0,0.0,0.0,2.0,1.0,3,0.8375,0.1625,Sampras P.,13,0.8375,Stoliarov A.,107,0.1625,2.0,1.0,1.0,2.0,16.0,13.0,13.0,16.0,2


In [47]:
tennis_df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3',
       'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets', 'Tournament Rank',
       'real_prob_W', 'real_prob_L', 'HR_name', 'HR_rank', 'HR_prob',
       'LR_name', 'LR_rank', 'LR_prob', 'HR_Wsets', 'LR_Wsets', 'HR_Lsets',
       'LR_Lsets', 'HR_Wgames', 'LR_Wgames', 'HR_Lgames', 'LR_Lgames',
       'Match_szn'],
      dtype='object')

In [48]:
drop_cols = [ 'W1', 'L1', 'W2', 'L2', 'W3', 'L3',
       'W4', 'L4', 'W5', 'L5']

tennis_df = tennis_df.drop(drop_cols, axis=1)

In [49]:
tennis_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Winner,Loser,WRank,LRank,Wsets,Lsets,Tournament Rank,real_prob_W,real_prob_L,HR_name,HR_rank,HR_prob,LR_name,LR_rank,LR_prob,HR_Wsets,LR_Wsets,HR_Lsets,LR_Lsets,HR_Wgames,LR_Wgames,HR_Lgames,LR_Lgames,Match_szn
0,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Johansson T.,Pretzsch A.,7,133,2.0,0.0,3,0.8775,0.1225,Johansson T.,7,0.8775,Pretzsch A.,133,0.1225,2.0,0.0,0.0,2.0,13.0,6.0,6.0,13.0,2
1,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Kafelnikov Y.,Bjorkman J.,6,56,2.0,0.0,3,0.6560,0.3440,Kafelnikov Y.,6,0.6560,Bjorkman J.,56,0.3440,2.0,0.0,0.0,2.0,13.0,8.0,8.0,13.0,2
2,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Koubek S.,Moya C.,41,20,2.0,0.0,3,0.3440,0.6560,Moya C.,20,0.6560,Koubek S.,41,0.3440,0.0,2.0,2.0,0.0,6.0,12.0,12.0,6.0,2
3,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Pavel A.,Stepanek R.,21,150,2.0,0.0,3,0.6155,0.3845,Pavel A.,21,0.6155,Stepanek R.,150,0.3845,2.0,0.0,0.0,2.0,13.0,10.0,10.0,13.0,2
4,32,Halle,Gerry Weber Open,2002-06-10,International,Outdoor,Grass,Sampras P.,Stoliarov A.,13,107,2.0,1.0,3,0.8375,0.1625,Sampras P.,13,0.8375,Stoliarov A.,107,0.1625,2.0,1.0,1.0,2.0,16.0,13.0,13.0,16.0,2


In [50]:
tennis_df_historical = tennis_df.copy()

In [75]:
# creates columns with information by rank instead of just who won
def generate_set_information(df, player_name):
    sets_won = 0
    sets_lost = 0

    # making sure we are only looking at dataframes containing
    # out player of interest
    assert((
        len(df[df['LR_name']==player_name]) +
        len(df[df['HR_name']==player_name]))==
        len(df)
        )
    # print(player_name, "History:")
    # display(df)

    higher_rank_matches = df[df['HR_name']==player_name]
    sets_won += sum(higher_rank_matches['HR_Wsets'])
    sets_lost += sum(higher_rank_matches['HR_Lsets'])

    # display(higher_rank_matches)

    lower_rank_matches = df[df['LR_name']==player_name]
    sets_won += sum(lower_rank_matches['LR_Wsets'])
    sets_lost += sum(lower_rank_matches['LR_Lsets'])

    # display(lower_rank_matches)

    total_sets = sets_won + sets_lost

    set_difference = sets_won - sets_lost
    total_matches = len(df)

    try:
        set_dif_per_match = set_difference/total_matches
    except:
        set_dif_per_match = np.nan

    return set_dif_per_match



In [76]:
# creates columns with information by rank instead of just who won
def generate_game_information(df, player_name):
    games_won = 0
    games_lost = 0

    # making sure we are only looking at dataframes containing
    # out player of interest
    assert((
        len(df[df['LR_name']==player_name]) +
        len(df[df['HR_name']==player_name]))==
        len(df)
        )
    # print(player_name, "History:")
    # display(df)

    higher_rank_matches = df[df['HR_name']==player_name]
    games_won += sum(higher_rank_matches['HR_Wgames'])
    games_lost += sum(higher_rank_matches['HR_Lgames'])

    # display(higher_rank_matches)

    lower_rank_matches = df[df['LR_name']==player_name]
    games_won += sum(lower_rank_matches['LR_Wgames'])
    games_lost += sum(lower_rank_matches['LR_Lgames'])

    # display(lower_rank_matches)

    total_games = games_won + games_lost

    games_difference = games_won - games_lost
    total_matches = len(df)


    try:
        games_dif_per_match = games_difference/total_matches
    except:
        games_dif_per_match = np.nan

    # display(df)
    # print("total_matches", total_matches)
    # print("games_dif_per_match", games_dif_per_match)

    return games_dif_per_match

In [77]:
# queries into whole dataset to filter so we're only looking at matches of a player before the match of interest
def historical_match_data(player_name, date_of_match):
#     isolated all matches a player has played in (whole dataset)
    player_rank_history = tennis_df_historical.loc[(tennis_df_historical['Winner'] == player_name) | (tennis_df_historical['Loser'] == player_name)]
#     filter this so we're only looking at historical matches

    player_rank_history = player_rank_history.loc[player_rank_history['Date'] < date_of_match]

    return player_rank_history

In [78]:
# returns weighted wins with the following formula:
# Rank = Wins * Wins / (Wins + Losses)
# https://stackoverflow.com/questions/7509219/weighted-win-percentage-by-number-of-games-played
def convert_wl_to_win_ratio(df, player_name):
    
    winning_matches = len(df.loc[df['Winner']==player_name])
    losing_matches = len(df.loc[df['Loser']==player_name])
    total_matches = winning_matches+losing_matches
    assert(winning_matches + losing_matches == len(df))

    win_win_ratio = (winning_matches*winning_matches)/total_matches
    return win_win_ratio

In [79]:
# returns weighted wins with the following formula:
# Rank = (Wins + constant * Average Win % of all players) / (Wins + Losses + constant)
# https://stackoverflow.com/questions/7509219/weighted-win-percentage-by-number-of-games-played

def convert_wl_to_win_ratio(df, player_name, constant, av_win_pct):
    
    winning_matches = len(df.loc[df['Winner']==player_name])
    losing_matches = len(df.loc[df['Loser']==player_name])
    total_matches = winning_matches+losing_matches
    assert(winning_matches + losing_matches == len(df))

    ratio = ((winning_matches+constant)*av_win_pct)/(winning_matches+losing_matches+constant)

    return ratio

In [80]:
# calculates number historic championships
def number_historic_championchips(df, player_name):

    # final round match
    final_matches = df.loc[df['Tournament Rank']==1]
    winner_final_match = final_matches.loc[final_matches['Winner']==player_name]
    num_championchips = len(winner_final_match)
    # print("HISTORICAL CHAMPIONCHIPS")
    # display(winner_final_match)
    # print(num_championchips)
    return num_championchips

In [81]:
# returns winning percentage for given df
# this function should only be called after historical_match_data()
def winning_percentage(player_df, player_name):
    
#     find where player wins, loses, then get percentage
    appears_winner = len(player_df.loc[player_df['Winner']==player_name])
    appears_loser = len(player_df.loc[player_df['Loser']==player_name])
    assert(appears_winner + appears_loser == len(player_df))
    winning_percentage = appears_winner/len(player_df)
    
    return winning_percentage

In [82]:
# returns multiple winning percentages: based on court type (outdoor, indoor), surface (grass, clay, etc), and series (Grand slam, etc.)
def winning_percentage_stats(player_history, player_name, court_type, 
surface_type, series_type, season_type, tournament_type):

#     isolate court type
    player_court_type = player_history.loc[player_history['Court']==court_type]

    try:
        court_type_winning_pct = winning_percentage(
        player_df=player_court_type, 
        player_name=player_name)
    except:
        court_type_winning_pct = np.nan

#     assert statements to check we isolated correctly
#     assert(len(set(player_court_type['Court']))==1)
#     assert(set(player_court_type['Court']).pop()==court_type)
    
#     repeat same as above for surface and series
    player_surface_type = player_history.loc[player_history['Surface']==surface_type]

    try: surface_type_winning_pct = winning_percentage(
        player_df=player_surface_type, 
        player_name=player_name)
    except:
        surface_type_winning_pct = np.nan

#     assert(len(set(player_surface_type['Surface']))==1)
#     assert(set(player_surface_type['Surface']).pop()==surface_type)

    player_series_type = player_history.loc[player_history['Series']==series_type]

    try:
        series_type_winning_pct = winning_percentage(
        player_df=player_series_type, 
        player_name=player_name)
    except:
        series_type_winning_pct = np.nan


    player_season_type = player_history.loc[player_history['Match_szn']==season_type]

    try:
        szn_type_winning_pct = winning_percentage(
        player_df=player_season_type, 
        player_name=player_name)
    except:
        szn_type_winning_pct = np.nan

    player_tournament = player_history.loc[player_history['Tournament']==tournament_type]

    try: tournament_win_pct = winning_percentage(
        player_df=player_tournament, 
        player_name=player_name)
    except:
        tournament_win_pct = np.nan

#     assert(len(set(player_series_type['Series']))==1)
#     assert(set(player_series_type['Series']).pop()==series_type)
    
    # set values for filtering
    # if ((len(player_court_type) < 10) 
    # or (len(player_surface_type) < 10) 
    # or (len(player_series_type) < 10)
    # or (len(player_season_type) < 10)
    # or (len(player_tournament) < 4)):
    #     return None
        # return (None, None, None, None, None)
    # else:
    #     put the above dataframes into the winning_percentage() function to calculate winning percentage
    #     based on these factors


    
       # print(player_name, court_type, surface_type, series_type)
    #     display(player_court_type)
    #     display(player_surface_type)
    #     display(player_series_type)

    #     return results as a tuple
    return (court_type_winning_pct, 
        surface_type_winning_pct, 
        series_type_winning_pct, 
        szn_type_winning_pct,
        tournament_win_pct)

In [83]:
# find matches when they play head to head
def head_to_head_performance(player_history, player_name, opponent_name):
    
    head_to_head_matches = player_history[
        (player_history['HR_name']==opponent_name) | 
        (player_history['LR_name']==opponent_name)]

    if len(head_to_head_matches) < 1:
        # print("No matches")
        return np.nan
    
    # win_pct_h2h = winning_percentage(
    #     player_df=head_to_head_matches, 
    #     player_name=player_name)

# takes the difference of amount of games won vs lost, then divides by num matches
# calculates the average margin of victory (terms of number of games) per match
    player_head_to_head_games_pct = generate_game_information(
        df=head_to_head_matches, 
        player_name=player_name)

    # print("Head to head:")
    # display(head_to_head_matches)
    # print("win pct:", win_pct_h2h)
    return player_head_to_head_games_pct

    

In [91]:
# this looks at performance as higher or lower seed
def performance_as_rank(df, player_name, rank_comparison):
    # print("player history")
    # display(df)
    results_df = None
    assert(rank_comparison=='higher' or rank_comparison=='lower')
    if rank_comparison=='higher':
        results_df = df[df['HR_name']==player_name]

        # no games where player was higher seed
        if len(results_df)==0:
            return np.nan
        else:
            assert(len(results_df['HR_name'].unique())==1)
    else:
        assert(rank_comparison=='lower')
        results_df = df[df['LR_name']==player_name]

        if len(results_df)==0:
            return np.nan
        else:
            assert(len(results_df['LR_name'].unique())==1)
            

# calculates the average margin of victory (terms of number of games) per match
    rank_game_dif_pct = generate_game_information(
        df=results_df, 
        player_name=player_name)

    return rank_game_dif_pct
    


In [92]:
# returns a bunch of stats for a player
def player_stats(name, rank_comparison, opponent_name, rank, prob, last_n_games, last_n_days, match_date, 
court_type, surface_type, series_type, season_type, tournament):
    
#     get player historical data, calculate days since last match
#     player_history = historical_match_data(player_name=name)
    player_history = historical_match_data(
        player_name=name,
        date_of_match=match_date
    )

    # if player has less than 5 matches, don't count in dataset
    if len(player_history) < 5:
        return None
        
    else:
    #     .date()
        # player_last_match = player_history.iloc[-1]['Date']
        # #print(match_date, player_last_match)
        # player_days_since_last_match = (match_date - player_last_match).days

        all_time_win_pct = winning_percentage(
            player_df=player_history,
            player_name=name)

        sets_dif_per_match = generate_set_information(
            df=player_history, 
            player_name=name)

        games_dif_per_match =  generate_game_information(
            df=player_history, 
            player_name=name)

        rank_performance_game_dif = performance_as_rank(
            df=player_history, 
            player_name=name, 
            rank_comparison=rank_comparison)

        num_champs = number_historic_championchips(
            df=player_history, 
            player_name=name)


        h2h_games_dif_per_match = head_to_head_performance(
            player_history=player_history, 
            player_name=name, 
            opponent_name=opponent_name)

        career_matches = len(player_history)
        # print("Player History")
        # display(player_history)

        last_n_days_dt = datetime.timedelta(last_n_days)
        n_days_before_match = match_date - last_n_days_dt

        # player_historical_match_days = historical_match_data(
        #     player_name=name, 
        #     date_of_match=match_date, 
        #     start_date=n_days_before_match
        # )

        player_historical_match_days = player_history.loc[
            (player_history['Date'] < match_date) & 
            ((player_history['Date'] > n_days_before_match))
        ]


        amount_matches_n_days_before = len(player_historical_match_days)
        match_per_day_pecentage = amount_matches_n_days_before/last_n_days

        # print(match_date, last_n_days, last_n_days_dt, n_days_before_match)
        # display(player_historical_match_days, match_per_day_pecentage)

        

    #     find player last n games (to calculate win percentage for recent matches)
        player_last_n_games = player_history[-last_n_games:]

    #     get recent player win pct
        recent_win_pct = winning_percentage(
            player_df=player_last_n_games,
            player_name=name)


        recent_sets_dif_per_match = generate_set_information(
            df=player_last_n_games, 
            player_name=name)

        recent_games_dif_per_match =  generate_game_information(
            df=player_last_n_games, 
            player_name=name)

    #     get player win percentage on the court the match is on, the sruface type of the match, and the series type
        (player_court_type_win_pct, 
        player_surface_type_win_pct, 
        player_series_type_win_pct, 
        szn_type_winning_pct, 
        tournament_win_pct) = winning_percentage_stats(
            player_history=player_history, 
            player_name=name, 
            court_type=court_type,
            surface_type=surface_type,
            series_type=series_type,
            season_type=season_type,
            tournament_type=tournament,
        )

    #     return all this info for the player
        ret_dict = {
                'name':name,
                'rank':rank,
                'prob':prob,

                'sets_dif_per_match':sets_dif_per_match,
                'games_dif_per_match': games_dif_per_match,
                'rank_performance_game_dif':rank_performance_game_dif,
                
                'num_champs':num_champs,
                # number finals appearances?
                'h2h_games_dif_per_match':h2h_games_dif_per_match,
                'career_matches':career_matches,
                'matches_per_day_rate': match_per_day_pecentage,
                'all_time_win_pct':all_time_win_pct,

                # totals within last n games (as of right now it is 20)
                'recent_win_pct': recent_win_pct,
                'recent_sets_dif_per_match': recent_sets_dif_per_match,
                'recent_games_dif_per_match':recent_games_dif_per_match,


                # all of these seem optional
                'court_type_win_pct':player_court_type_win_pct,
                'surface_type_win_pct':player_surface_type_win_pct,
                'series_type_win_pct':player_series_type_win_pct,
                'szn_type_win_pct':szn_type_winning_pct,
                'tournament_win_pct':tournament_win_pct,
            }

        return ret_dict

In [93]:
from datetime import date

# this creates the dataset we can use for ML
# with feature engineering

def ML_df_function(df, start_row=None, end_row=None):

#     set up return DF with our new formatted data
#     to find what columns to put for final_df:
#     display(returned_row.keys())
    final_columns = ['Location', 'Date', 'Series', 'Court', 'Surface', 'Szn', 'ATP', 'HR_win?', 'HR_name', 'LR_name', 'higher_ranked_rank', 'higher_ranked_prob', 'higher_ranked_sets_dif_per_match', 'higher_ranked_games_dif_per_match', 'higher_ranked_rank_performance_game_dif', 'higher_ranked_num_champs', 'higher_ranked_h2h_games_dif_per_match', 'higher_ranked_career_matches', 'higher_ranked_matches_per_day_rate', 'higher_ranked_all_time_win_pct', 'higher_ranked_recent_win_pct', 'higher_ranked_recent_sets_dif_per_match', 'higher_ranked_recent_games_dif_per_match', 'higher_ranked_court_type_win_pct', 'higher_ranked_surface_type_win_pct', 'higher_ranked_series_type_win_pct', 'higher_ranked_szn_type_win_pct', 'higher_ranked_tournament_win_pct', 'lower_ranked_rank', 'lower_ranked_prob', 'lower_ranked_sets_dif_per_match', 'lower_ranked_games_dif_per_match', 'lower_ranked_rank_performance_game_dif', 'lower_ranked_num_champs', 'lower_ranked_h2h_games_dif_per_match', 'lower_ranked_career_matches', 'lower_ranked_matches_per_day_rate', 'lower_ranked_all_time_win_pct', 'lower_ranked_recent_win_pct', 'lower_ranked_recent_sets_dif_per_match', 'lower_ranked_recent_games_dif_per_match', 'lower_ranked_court_type_win_pct', 'lower_ranked_surface_type_win_pct', 'lower_ranked_series_type_win_pct', 'lower_ranked_szn_type_win_pct', 'lower_ranked_tournament_win_pct', 'diff_rank', 'diff_prob', 'diff_sets_dif_per_match', 'diff_games_dif_per_match', 'diff_rank_performance_game_dif', 'diff_num_champs', 'diff_h2h_games_dif_per_match', 'diff_career_matches', 'diff_matches_per_day_rate', 'diff_all_time_win_pct', 'diff_recent_win_pct', 'diff_recent_sets_dif_per_match', 'diff_recent_games_dif_per_match', 'diff_court_type_win_pct', 'diff_surface_type_win_pct', 'diff_series_type_win_pct', 'diff_szn_type_win_pct', 'diff_tournament_win_pct']
    final_df = pd.DataFrame(columns = final_columns)


    num_rows = end_row-start_row
    print(f"{num_rows} total rows in dataset")
    current_row = 0

#     tells us progress of processing
    for i in range(start_row,end_row):

        current_row += 1
        if (current_row % 5000==0):
            print(f"{current_row} rows processed.")

#         create dictionary to return the new row
        returned_row = {}
        returned_row['Location'] = df.loc[i,'Location']
        returned_row['Date'] = df.loc[i,'Date']
        returned_row['Series'] = df.loc[i,'Series']
        returned_row['Court'] = df.loc[i,'Court']
        returned_row['Surface'] = df.loc[i,'Surface']
        returned_row['Szn'] = df.loc[i,'Match_szn']
        returned_row['ATP'] = df.loc[i,'ATP']


#         DISPLAY original df row
        # display(df.iloc[i].to_frame().T)
        # display(df.loc[i,'HR_name'])

#         get date of the match
        date_of_match = df.loc[i,'Date']

        # min amount of games to be considered
        last_n_games = 20
        # days previous to calculate various recentcy stats
        last_n_days = 180
        
        higher_ranked_player_stats = player_stats(
            name=df.loc[i,'HR_name'],
            rank_comparison = 'higher',
            opponent_name=df.loc[i,'LR_name'],
            rank=df.loc[i,'HR_rank'],
            prob=df.loc[i,'HR_prob'],
            last_n_games=last_n_games,
            last_n_days=last_n_days,
            match_date = df.loc[i,'Date'],
            court_type=df.loc[i,'Court'],
            surface_type=df.loc[i,'Surface'],
            series_type=df.loc[i,'Series'], 
            season_type=df.loc[i,'Match_szn'], 
            tournament=df.loc[i,'Tournament']
        )

        lower_ranked_player_stats = player_stats(
            name=df.loc[i,'LR_name'],
            rank_comparison = 'lower',
            opponent_name=df.loc[i,'HR_name'],
            rank=df.loc[i,'LR_rank'],
            prob=df.loc[i,'LR_prob'],
            last_n_games=last_n_games,
            last_n_days=last_n_days,
            match_date = df.loc[i,'Date'],
            court_type=df.loc[i,'Court'],
            surface_type=df.loc[i,'Surface'],
            series_type=df.loc[i,'Series'],
            season_type=df.loc[i,'Match_szn'], 
            tournament=df.loc[i,'Tournament']
        )
        
        if (higher_ranked_player_stats == None) or (lower_ranked_player_stats == None):
            i += 1
        else:
    #         display(higher_ranked_player_stats)
    #         display(lower_ranked_player_stats)

    #         for each stat, append this to the returned fr (row to be 
    #         added to new ML df)


            hr_name = higher_ranked_player_stats['name']
    #         print(hr_name)
            winner = df.loc[i,'Winner']
    #         print(winner)

            if hr_name == winner:
                returned_row['HR_win?']=1
            else:
                returned_row['HR_win?']=0

    #         print(hr_name==winner)
    #         deletes name so we are only taking diff on numeric columns

            returned_row['HR_name'] = higher_ranked_player_stats['name']
            returned_row['LR_name'] = lower_ranked_player_stats['name']

            del higher_ranked_player_stats['name']
            del lower_ranked_player_stats['name']


            value_keys = list(higher_ranked_player_stats.keys())
    #         display(value_keys)
    #         print("VALUES")
            high_rank_list = list(higher_ranked_player_stats.values())
            low_rank_list = list(lower_ranked_player_stats.values())

            for key, stat in zip(value_keys, high_rank_list):
                returned_row[f'higher_ranked_{key}'] = stat

            for key, stat in zip(value_keys, low_rank_list):
                returned_row[f'lower_ranked_{key}'] = stat

            # display(high_rank_list)
            # display(low_rank_list)
            diffs = [a_i - b_i for a_i, b_i in zip(high_rank_list, low_rank_list)]
    #         print("diffs")
    #         display(diffs)


            for key, diff in zip(value_keys, diffs):
                returned_row[f'diff_{key}'] = diff


            # display(returned_row.keys())
            final_df = final_df.append(returned_row, ignore_index=True)
    return final_df


In [94]:
tennis_df = tennis_df.reset_index(drop=True)

In [95]:
tennis_df.tail()

,ATP,Location,Tournament,Date,Series,Court,Surface,Winner,Loser,WRank,LRank,Wsets,Lsets,Tournament Rank,real_prob_W,real_prob_L,HR_name,HR_rank,HR_prob,LR_name,LR_rank,LR_prob,HR_Wsets,LR_Wsets,HR_Lsets,LR_Lsets,HR_Wgames,LR_Wgames,HR_Lgames,LR_Lgames,Match_szn
47897,58,Vienna,Vienna Open,2021-10-29,ATP500,Indoor,Hard,Tiafoe F.,Schwartzman D.,49,16,2.0,0.0,3,0.4265,0.5735,Schwartzman D.,16,0.5735,Tiafoe F.,49,0.4265,0.0,2.0,2.0,0.0,10.0,13.0,13.0,10.0,3
47898,58,Vienna,Vienna Open,2021-10-29,ATP500,Indoor,Hard,Sinner J.,Ruud C.,11,8,2.0,0.0,3,0.6905,0.3095,Ruud C.,8,0.3095,Sinner J.,11,0.6905,0.0,2.0,2.0,0.0,6.0,13.0,13.0,6.0,3
47899,58,Vienna,Vienna Open,2021-10-30,ATP500,Indoor,Hard,Zverev A.,Alcaraz C.,4,42,2.0,0.0,3,0.7570,0.2430,Zverev A.,4,0.7570,Alcaraz C.,42,0.2430,2.0,0.0,0.0,2.0,12.0,6.0,6.0,12.0,3
47900,58,Vienna,Vienna Open,2021-10-30,ATP500,Indoor,Hard,Tiafoe F.,Sinner J.,49,11,2.0,1.0,3,0.2250,0.7750,Sinner J.,11,0.7750,Tiafoe F.,49,0.2250,1.0,2.0,2.0,1.0,13.0,16.0,16.0,13.0,3
47901,58,Vienna,Vienna Open,2021-10-31,ATP500,Indoor,Hard,Zverev A.,Tiafoe F.,4,49,2.0,0.0,1,0.8055,0.1945,Zverev A.,4,0.8055,Tiafoe F.,49,0.1945,2.0,0.0,0.0,2.0,13.0,9.0,9.0,13.0,3


In [96]:
ML_dataframe = ML_df_function(
    df=tennis_df, 
    start_row=0, end_row=len(tennis_df)
    # start_row=40000, end_row=40010
)


# Features:

# who won, along with all the drf features
# HRwin? 0 1

#     last 10 winning
#     number days sine last match
#     surface type
#     outdoor vs indoor
#     winning percentage in type of series
#     winning percentage in round
#     How much rank is chancged before match

47902 total rows in dataset
5000 rows processed.
10000 rows processed.
15000 rows processed.
20000 rows processed.
25000 rows processed.
30000 rows processed.
35000 rows processed.
40000 rows processed.
45000 rows processed.


In [99]:
ML_dataframe.tail()

,Location,Date,Series,Court,Surface,Szn,ATP,HR_win?,HR_name,LR_name,higher_ranked_rank,higher_ranked_prob,higher_ranked_sets_dif_per_match,higher_ranked_games_dif_per_match,higher_ranked_rank_performance_game_dif,higher_ranked_num_champs,higher_ranked_h2h_games_dif_per_match,higher_ranked_career_matches,higher_ranked_matches_per_day_rate,higher_ranked_all_time_win_pct,higher_ranked_recent_win_pct,higher_ranked_recent_sets_dif_per_match,higher_ranked_recent_games_dif_per_match,higher_ranked_court_type_win_pct,higher_ranked_surface_type_win_pct,higher_ranked_series_type_win_pct,higher_ranked_szn_type_win_pct,higher_ranked_tournament_win_pct,lower_ranked_rank,lower_ranked_prob,lower_ranked_sets_dif_per_match,lower_ranked_games_dif_per_match,lower_ranked_rank_performance_game_dif,lower_ranked_num_champs,lower_ranked_h2h_games_dif_per_match,lower_ranked_career_matches,lower_ranked_matches_per_day_rate,lower_ranked_all_time_win_pct,lower_ranked_recent_win_pct,lower_ranked_recent_sets_dif_per_match,lower_ranked_recent_games_dif_per_match,lower_ranked_court_type_win_pct,lower_ranked_surface_type_win_pct,lower_ranked_series_type_win_pct,lower_ranked_szn_type_win_pct,lower_ranked_tournament_win_pct,diff_rank,diff_prob,diff_sets_dif_per_match,diff_games_dif_per_match,diff_rank_performance_game_dif,diff_num_champs,diff_h2h_games_dif_per_match,diff_career_matches,diff_matches_per_day_rate,diff_all_time_win_pct,diff_recent_win_pct,diff_recent_sets_dif_per_match,diff_recent_games_dif_per_match,diff_court_type_win_pct,diff_surface_type_win_pct,diff_series_type_win_pct,diff_szn_type_win_pct,diff_tournament_win_pct
43465,Vienna,2021-10-29,ATP500,Indoor,Hard,3,58,0,Schwartzman D.,Tiafoe F.,16,0.5735,0.297059,1.179412,2.768182,4,4.500000,340,0.183333,0.567647,0.75,0.90,2.45,0.600000,0.577381,0.576923,0.613636,1.0,49,0.4265,-0.102041,-0.428571,-1.017241,1,-4.500000,196,0.150000,0.454082,0.60,0.45,1.00,0.466667,0.468085,0.406250,0.476190,1.0,-33,0.147,0.399100,1.607983,3.785423,3,9.000000,144,0.033333,0.113565,0.15,0.45,1.45,0.133333,0.109296,0.170673,0.137446,0.0
43466,Vienna,2021-10-29,ATP500,Indoor,Hard,3,58,0,Ruud C.,Sinner J.,8,0.3095,0.378378,1.689189,3.714286,6,-4.000000,148,0.244444,0.621622,0.80,0.90,2.90,0.578947,0.468085,0.600000,0.593750,1.0,11,0.6905,0.537736,1.594340,-0.555556,5,4.000000,106,0.200000,0.650943,0.80,1.10,3.90,0.750000,0.704225,0.750000,0.767442,1.0,-3,-0.381,-0.159357,0.094850,4.269841,1,-8.000000,42,0.044444,-0.029322,0.00,-0.20,-1.00,-0.171053,-0.236140,-0.150000,-0.173692,0.0
43467,Vienna,2021-10-30,ATP500,Indoor,Hard,3,58,1,Zverev A.,Alcaraz C.,4,0.7570,0.747500,2.047500,3.199324,16,8.000000,400,0.205556,0.685000,0.85,1.60,4.75,0.648649,0.681034,0.673077,0.673077,1.0,42,0.2430,0.400000,1.700000,0.518519,1,-8.000000,40,0.138889,0.625000,0.75,0.75,2.45,1.000000,0.652174,0.571429,0.857143,1.0,-38,0.514,0.347500,0.347500,2.680806,15,16.000000,360,0.066667,0.060000,0.10,0.85,2.30,-0.351351,0.028861,0.101648,-0.184066,0.0
43468,Vienna,2021-10-30,ATP500,Indoor,Hard,3,58,0,Sinner J.,Tiafoe F.,11,0.7750,0.551402,1.644860,3.180328,5,2.000000,107,0.205556,0.654206,0.80,1.15,3.95,0.756757,0.708333,0.761905,0.772727,1.0,49,0.2250,-0.091371,-0.411168,-0.982906,1,-2.000000,197,0.155556,0.456853,0.65,0.70,1.50,0.483871,0.471831,0.424242,0.488372,1.0,-38,0.550,0.642772,2.056027,4.163234,4,4.000000,-90,0.050000,0.197353,0.15,0.45,2.45,0.272886,0.236502,0.337662,0.284355,0.0
43469,Vienna,2021-10-31,ATP500,Indoor,Hard,3,58,1,Zverev A.,Tiafoe F.,4,0.8055,0.750623,2.057357,3.208754,16,4.666667,401,0.211111,0.685786,0.85,1.55,4.60,0.653333,0.682403,0.676190,0.676190,1.0,49,0.1945,-0.085859,-0.393939,-0.949153,1,-4.666667,198,0.161111,0.459596,0.70,0.85,1.80,0.500000,0.475524,0.441176,0.500000,1.0,-45,0.611,0.836482,2.451296,4.157907,15,9.333333,203,0.050000,0.226190,0.15,0.70,2.80,0.153333,0.206879,0.235014,0.176190,0.0


In [101]:
# check missing vals in cols:
pd.set_option("display.max_rows", None)
percent_missing = ML_dataframe.isnull().sum() * 100 / len(ML_dataframe)
missing_value_df = pd.DataFrame({'column_name': ML_dataframe.columns,
                            'percent_missing': percent_missing})
display(missing_value_df)

,column_name,percent_missing
Location,Location,0.000000
Date,Date,0.000000
Series,Series,0.000000
Court,Court,0.000000
Surface,Surface,0.000000
Szn,Szn,0.000000
ATP,ATP,0.000000
HR_win?,HR_win?,0.000000
HR_name,HR_name,0.000000
LR_name,LR_name,0.000000


In [102]:
HR_win_col = ML_dataframe.pop('HR_win?')
ML_dataframe.insert(len(ML_dataframe.columns), 'HR_win?', HR_win_col)

In [103]:
ML_dataframe.head()

,Location,Date,Series,Court,Surface,Szn,ATP,HR_name,LR_name,higher_ranked_rank,higher_ranked_prob,higher_ranked_sets_dif_per_match,higher_ranked_games_dif_per_match,higher_ranked_rank_performance_game_dif,higher_ranked_num_champs,higher_ranked_h2h_games_dif_per_match,higher_ranked_career_matches,higher_ranked_matches_per_day_rate,higher_ranked_all_time_win_pct,higher_ranked_recent_win_pct,higher_ranked_recent_sets_dif_per_match,higher_ranked_recent_games_dif_per_match,higher_ranked_court_type_win_pct,higher_ranked_surface_type_win_pct,higher_ranked_series_type_win_pct,higher_ranked_szn_type_win_pct,higher_ranked_tournament_win_pct,lower_ranked_rank,lower_ranked_prob,lower_ranked_sets_dif_per_match,lower_ranked_games_dif_per_match,lower_ranked_rank_performance_game_dif,lower_ranked_num_champs,lower_ranked_h2h_games_dif_per_match,lower_ranked_career_matches,lower_ranked_matches_per_day_rate,lower_ranked_all_time_win_pct,lower_ranked_recent_win_pct,lower_ranked_recent_sets_dif_per_match,lower_ranked_recent_games_dif_per_match,lower_ranked_court_type_win_pct,lower_ranked_surface_type_win_pct,lower_ranked_series_type_win_pct,lower_ranked_szn_type_win_pct,lower_ranked_tournament_win_pct,diff_rank,diff_prob,diff_sets_dif_per_match,diff_games_dif_per_match,diff_rank_performance_game_dif,diff_num_champs,diff_h2h_games_dif_per_match,diff_career_matches,diff_matches_per_day_rate,diff_all_time_win_pct,diff_recent_win_pct,diff_recent_sets_dif_per_match,diff_recent_games_dif_per_match,diff_court_type_win_pct,diff_surface_type_win_pct,diff_series_type_win_pct,diff_szn_type_win_pct,diff_tournament_win_pct,HR_win?
0,Stuttgart,2002-07-15,International,Outdoor,Clay,2,41,El Aynaoui Y.,Youzhny M.,17,0.6560,0.500000,3.000000,3.00,0,NaN,6,0.033333,0.666667,0.666667,0.500000,3.000000,0.666667,0.800000,0.800000,0.666667,NaN,62,0.3440,0.333333,1.50,1.50,0,NaN,6,0.033333,0.666667,0.666667,0.333333,1.50,0.666667,0.500000,0.666667,0.666667,NaN,-45,0.312,0.166667,1.500000,1.50,0,NaN,0,0.000000,0.000000,0.000000,0.166667,1.500000,0.000000,0.300000,0.133333,0.000000,NaN,0
1,Stuttgart,2002-07-15,International,Outdoor,Clay,2,41,Canas G.,Youzhny M.,19,0.6750,0.800000,4.000000,4.00,0,NaN,5,0.027778,0.600000,0.600000,0.800000,4.000000,0.600000,0.666667,0.666667,0.600000,NaN,62,0.3250,0.333333,1.50,1.50,0,NaN,6,0.033333,0.666667,0.666667,0.333333,1.50,0.666667,0.500000,0.666667,0.666667,NaN,-43,0.350,0.466667,2.500000,2.50,0,NaN,-1,-0.005556,-0.066667,-0.066667,0.466667,2.500000,-0.066667,0.166667,0.000000,-0.066667,NaN,0
2,Kitzbuhel,2002-07-22,International Gold,Outdoor,Clay,2,43,Schuettler R.,Davydenko N.,26,0.6155,1.000000,3.600000,2.75,0,NaN,5,0.027778,0.600000,0.600000,1.000000,3.600000,0.600000,0.000000,NaN,0.600000,NaN,93,0.3845,-0.600000,-2.20,-3.75,0,NaN,5,0.027778,0.400000,0.400000,-0.600000,-2.20,0.400000,0.500000,NaN,0.400000,NaN,-67,0.231,1.600000,5.800000,6.50,0,NaN,0,0.000000,0.200000,0.200000,1.600000,5.800000,0.200000,-0.500000,NaN,0.200000,NaN,0
3,Kitzbuhel,2002-07-22,International Gold,Outdoor,Clay,2,43,Calleri A.,Sa A.,47,0.6560,0.571429,2.571429,4.00,0,NaN,7,0.038889,0.571429,0.571429,0.571429,2.571429,0.571429,0.666667,NaN,0.571429,NaN,57,0.3440,0.750000,1.75,-0.60,0,NaN,8,0.044444,0.750000,0.750000,0.750000,1.75,0.750000,0.666667,NaN,0.750000,NaN,-10,0.312,-0.178571,0.821429,4.60,0,NaN,-1,-0.005556,-0.178571,-0.178571,-0.178571,0.821429,-0.178571,0.000000,NaN,-0.178571,NaN,0
4,Kitzbuhel,2002-07-22,International Gold,Outdoor,Clay,2,43,Chela J.I.,Davydenko N.,25,0.7210,1.166667,4.000000,3.60,1,NaN,6,0.033333,0.833333,0.833333,1.166667,4.000000,0.833333,0.833333,NaN,0.833333,NaN,93,0.2790,-0.600000,-2.20,-3.75,0,NaN,5,0.027778,0.400000,0.400000,-0.600000,-2.20,0.400000,0.500000,NaN,0.400000,NaN,-68,0.442,1.766667,6.200000,7.35,1,NaN,1,0.005556,0.433333,0.433333,1.766667,6.200000,0.433333,0.333333,NaN,0.433333,NaN,1


In [104]:
# display(ML_dataframe)

In [105]:
import numpy as np
np.percentile(tennis_df["LRank"], 90)

169.0

In [106]:
ML_dataframe.to_csv("complete_data.csv")

In [ ]:
# can either just focus on higher rank, and get info about his past and ooppondent's past
# or can take the difference in odds, which require us breaking each player into 2 rows, creating a regression on each, then subtracting the odds difference

# think the higher rank version is better option, focuses on just 1 regression and 1 decision

In [ ]:
# tennis_df.to_excel("tennis_data_2001_2021.xlsx") 